In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import scale, StandardScaler
from sklearn.datasets import load_boston
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.svm import SVC
from sklearn import datasets
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
data = pd.read_csv('data-logistic.csv', header = None)

data

,0,1,2
0,-1,-0.663827,-0.138526
1,1,1.994596,2.468025
2,-1,-1.247395,0.749425
3,1,2.309374,1.899836
4,1,0.849143,2.407750
5,1,1.454271,-0.665416
6,1,2.254227,2.263786
7,-1,-0.067580,1.469141
8,-1,-0.861961,-0.824856
9,1,0.699179,2.032488


In [3]:
Y_ = data.values[:,:1].T[0]

Y_

array([-1.,  1., -1.,  1.,  1.,  1.,  1., -1., -1.,  1., -1., -1., -1.,
       -1.,  1., -1., -1.,  1.,  1.,  1., -1.,  1., -1., -1., -1., -1.,
        1.,  1., -1., -1., -1., -1.,  1.,  1., -1., -1.,  1.,  1., -1.,
        1.,  1., -1.,  1., -1., -1.,  1.,  1., -1., -1.,  1.,  1.,  1.,
       -1.,  1., -1., -1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,
        1., -1.,  1., -1.,  1.,  1., -1.,  1., -1., -1., -1.,  1., -1.,
        1., -1.,  1., -1.,  1.,  1., -1.,  1.,  1., -1.,  1., -1., -1.,
       -1.,  1., -1., -1.,  1.,  1., -1., -1., -1.,  1.,  1., -1., -1.,
       -1.,  1.,  1.,  1.,  1., -1., -1.,  1.,  1.,  1., -1.,  1., -1.,
       -1.,  1., -1.,  1., -1.,  1., -1.,  1.,  1.,  1., -1.,  1.,  1.,
        1., -1., -1., -1.,  1., -1.,  1., -1.,  1., -1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1., -1., -1., -1., -1.,  1., -1.,
       -1.,  1., -1., -1., -1., -1.,  1.,  1.,  1.,  1., -1.,  1., -1.,
       -1.,  1., -1., -1., -1.,  1.,  1., -1.,  1., -1.,  1.,  1

In [4]:
X_ = data.values[:,1:]

X_

array([[-6.63826537e-01, -1.38525717e-01],
       [ 1.99459550e+00,  2.46802468e+00],
       [-1.24739492e+00,  7.49424636e-01],
       [ 2.30937425e+00,  1.89983556e+00],
       [ 8.49143314e-01,  2.40774982e+00],
       [ 1.45427095e+00, -6.65415709e-01],
       [ 2.25422743e+00,  2.26378585e+00],
       [-6.75795166e-02,  1.46914110e+00],
       [-8.61960913e-01, -8.24855578e-01],
       [ 6.99178929e-01,  2.03248761e+00],
       [-1.61479961e-01,  4.95461855e-01],
       [-1.47102261e-01,  4.19646539e-01],
       [ 8.02959560e-02, -1.15609708e+00],
       [ 1.72065543e+00, -1.08066154e+00],
       [ 1.57395765e+00,  1.78543356e+00],
       [ 1.16844518e+00, -6.45619241e-01],
       [-6.66273601e-02, -1.05974659e+00],
       [ 1.16335473e+00,  1.74588676e+00],
       [ 2.09696979e+00,  1.12963766e+00],
       [ 3.03570513e+00,  2.14283780e+00],
       [ 8.31852059e-01, -6.06776832e-01],
       [-1.29778002e+01, -4.57305579e+00],
       [-8.42603085e-01, -1.07628156e+00],
       [-4.

In [5]:
def sigm(x):
    
    return 1.0/(1+math.exp(-x))

In [6]:
def dist(a,b):
    
    return np.sqrt(np.square(a[0]-b[0]) + np.square(a[1]-b[1]))

In [13]:
def logregr(X,Y,k,w1,w2,C,eps,max_iter):
    
    for i in range(max_iter):
        
        w1n = w1 + k * np.mean(Y * X[:,0] * (1-(1.0 / (1+np.exp(-Y*(w1 * X[:,0] + w2 * X[:,1])))))) - k * C * w1
        
        w2n = w2 + k * np.mean(Y * X[:,1] * (1-(1.0 / (1+np.exp(-Y*(w1 * X[:,0] + w2 * X[:,1])))))) - k * C * w2
        
        if dist((w1n, w2n), (w1,w2)) < eps:
            break
        w1, w2 = w1n, w2n
        
    predicts = []
    
    print(w1,w2)
    
    for i in range(len(X)):
        
        t1 = w1*X[i, 0] + w2 * X[i,1]
        
        s = sigm(t1)
        
        predicts.append(s)
        
    return predicts

In [14]:
p0 = logregr(X_, Y_, 0.1, 0.0, 0.0, 0, 0.00001, 10000)

p0

0.28780442348832036 0.09198995719421853


[0.4492268093972114,
 0.6902069299853123,
 0.4279876518309828,
 0.6983417655900398,
 0.6144071988210699,
 0.5883949298676745,
 0.7020381293611153,
 0.5288919146723731,
 0.41971228672158617,
 0.595848163822862,
 0.4997757169456513,
 0.4990666474991173,
 0.4792020601445609,
 0.597678685671797,
 0.6495891880913306,
 0.5687843876203692,
 0.4708676103801266,
 0.6213830440032729,
 0.6698335882897946,
 0.7447507201942198,
 0.5457698422011846,
 0.01543224576369968,
 0.41544224344810093,
 0.44921321471442494,
 0.6127231049071752,
 0.5371070884674715,
 0.697730467950528,
 0.7586437413781593,
 0.47126983888725293,
 0.5872260785746587,
 0.4619693551523017,
 0.48523328241874397,
 0.7856953527975041,
 0.7499352728711124,
 0.5178699499177107,
 0.6608198469948112,
 0.7049395585432335,
 0.6673196286510747,
 0.5836182086508569,
 0.6960579230359982,
 0.6478499068347506,
 0.44015910426058724,
 0.5946892427353323,
 0.46013812209307914,
 0.5303392748741866,
 0.6420780364421801,
 0.8042418577357068,
 0.47228

In [15]:
p1 = logregr(X_, Y_, 0.1, 0.0, 0.0, 10, 0.00001, 10000)

p1

0.028561965517012657 0.024783655436404747


[0.4944018928628572,
 0.5294997596131825,
 0.4957364611836028,
 0.5282312264063188,
 0.5209692042685694,
 0.5062610234938691,
 0.530086123670424,
 0.5086192667796262,
 0.48873634735921273,
 0.5175783516200203,
 0.501916783313123,
 0.5015497064172023,
 0.4934106561990277,
 0.5055904064807847,
 0.5222864468105988,
 0.504342962292315,
 0.49295861490422793,
 0.519114967857427,
 0.5219583990092682,
 0.5348964378480556,
 0.5021802816514227,
 0.3813060191277351,
 0.4873175729995377,
 0.49206864696294605,
 0.5120371158328015,
 0.5039504784767476,
 0.5306951997434963,
 0.5410828222892188,
 0.4983936647080941,
 0.5135743504155265,
 0.49451140679822203,
 0.5024757990684602,
 0.54601061019731,
 0.5326244404113453,
 0.5073907944040208,
 0.5137868800163443,
 0.5341804475869174,
 0.5230126099933657,
 0.5072039793569557,
 0.5238331816056466,
 0.518282353204113,
 0.4970466177993987,
 0.5127026962326858,
 0.49238334886939933,
 0.5070339684402575,
 0.5197304288038219,
 0.543888364180494,
 0.4972682367553

In [18]:
round(roc_auc_score(Y_, p0), 3)

0.927

In [19]:
round(roc_auc_score(Y_, p1), 3)

0.936